In [8]:
## Low-Rank Adaptation
### Adapters can be loaded onto a pretrained model wiht (load_adapter())
### Set the active adapter weights with (set_adapter())
### Return base model (unload())

from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM

config = PeftConfig.from_pretrained("ybelkada/opt-350m-lora")
print(config.base_model_name_or_path)

model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)

lora_model = PeftModel.from_pretrained(model,"ybelkada/opt-350m-lora")

facebook/opt-350m


adapter_model.safetensors:   0%|          | 0.00/6.30M [00:00<?, ?B/s]

In [9]:
lora_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): OPTForCausalLM(
      (model): OPTModel(
        (decoder): OPTDecoder(
          (embed_tokens): Embedding(50272, 512, padding_idx=1)
          (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
          (project_out): Linear(in_features=1024, out_features=512, bias=False)
          (project_in): Linear(in_features=512, out_features=1024, bias=False)
          (layers): ModuleList(
            (0-23): 24 x OPTDecoderLayer(
              (self_attn): OPTAttention(
                (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
                (v_proj): lora.Linear(
                  (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=1024, out_features=16, bias=False

In [10]:
model

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 512, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
      (project_out): Linear(in_features=1024, out_features=512, bias=False)
      (project_in): Linear(in_features=512, out_features=1024, bias=False)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): lora.Linear(
              (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
              (lora_dropout): ModuleDict(
                (default): Dropout(p=0.05, inplace=False)
              )
              (lora_A): ModuleDict(
                (default): Linear(in_features=1024, out_features=16, bias=False)
              )
              (lora_B): ModuleDict(
                (default): Linear(in_features=16, out_features=1024, bias=Fa

In [11]:
## trl provides integration with LoRA adapters through PEFT library

# 1. Define the LoRA confuguration (rank, alpha, dropout)
# 2. Create the SFTTrainer with PEFT config
# 3. Train and save adapter weights

from peft import LoraConfig

rank = 5 # [4-32] smaller = more cmpression (dimension for LoRA update matrices)
lora_alpha = 8 # [2x(rank]) higher = stronger adaptation (scaling factor) "how much of the pretrained model's behavior is modified by newlwy added low-rank updates" modle output influence
lora_dropout = 0.05 # [0.05-1] helps prevent overfitting "probability"

peft_config = LoraConfig(
    r=rank,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropoput,
    bias=None,
    target_modules='all-linear', # which modules to apply LoRA to
    task_type='CAUSAL_LM', # task type for model arch 
    device_map='auto'
) 


False